In [97]:
import pennylane as qml




dev = qml.device('default.qubit', wires=5, shots=1)

In [98]:


@qml.qnode(dev)
def quantum_walk(input_node):

    for i in range(3):

        if input_node[i]==1:
            qml.PauliX(i)

    # Add a Hadamard gate to qubit 0
    qml.Hadamard(3)
    qml.Hadamard(4)
    # Perform a controlled-X gate on qubit 1, controlled by qubit 0
    qml.CNOT([4, 0])
    qml.PauliX(4)
    qml.CNOT([4, 1])
    qml.CNOT([3, 2])
    qml.MultiControlledX([4,3], 1)
    qml.PauliX(4)
    qml.MultiControlledX([4,3], 0)
    qml.PauliX(4)
    qml.MultiControlledX([4,3], 2)

    return qml.sample()

In [99]:
print(qml.draw(quantum_walk)([1,0,0]))

0: ──X─╭X────────────────╭X───────┤  Sample
1: ────│─────╭X────╭X────│────────┤  Sample
2: ────│─────│──╭X─│─────│─────╭X─┤  Sample
3: ──H─│─────│──╰●─├●────├●────├●─┤  Sample
4: ──H─╰●──X─╰●────╰●──X─╰●──X─╰●─┤  Sample


In [100]:
import numpy as np

def step(v):
    
    return quantum_walk(v)[:3]


In [101]:
pitch_mapping = {
    '000': 48,  # C3
    '001': 52,  # E3
    '010': 55,  # G3
    '011': 59,  # B3
    '100': 60,  # C4 (Middle C)
    '101': 64,  # E4
    '110': 67,  # G4
    '111': 71,  # B4
}



In [102]:
from midiutil import MIDIFile


track = 0
channel = 0
time = 0
duration = 1
tempo = 90
volume = 100

MyMIDI = MIDIFile(1)
MyMIDI.addTempo(track, time, tempo)

quarter_note = 1
half_note = 2

n = 16

def find_pitch(v):
    key = str(v).replace(' ', '').replace('[', '').replace(']', '')
    return pitch_mapping[key]

v = np.array([0,0,0])
for i in range(n):
    v = step(v)  
    pitch = find_pitch(v)  
    print(pitch) 
    print(f"Pitch: {pitch}, Type: {type(pitch)}") 
    MyMIDI.addNote(track, channel, pitch=pitch, duration=quarter_note, time=i, volume=volume)
 
    MyMIDI.addNote(track, channel, pitch=pitch, duration=quarter_note, time=i, volume=volume)

output_file_path = "pleasework.mid"
outf = open(output_file_path, 'wb')
MyMIDI.writeFile(outf)
outf.close()


48
Pitch: 48, Type: <class 'int'>
60
Pitch: 60, Type: <class 'int'>
67
Pitch: 67, Type: <class 'int'>
55
Pitch: 55, Type: <class 'int'>
55
Pitch: 55, Type: <class 'int'>
48
Pitch: 48, Type: <class 'int'>
52
Pitch: 52, Type: <class 'int'>
59
Pitch: 59, Type: <class 'int'>
52
Pitch: 52, Type: <class 'int'>
64
Pitch: 64, Type: <class 'int'>
60
Pitch: 60, Type: <class 'int'>
48
Pitch: 48, Type: <class 'int'>
48
Pitch: 48, Type: <class 'int'>
60
Pitch: 60, Type: <class 'int'>
64
Pitch: 64, Type: <class 'int'>
60
Pitch: 60, Type: <class 'int'>


In [103]:
import subprocess

def convert_midi_to_wav(midi_path, wav_path, soundfont_path):
    command = f'fluidsynth -ni {soundfont_path} {midi_path} -F {wav_path} -r 44100'
    subprocess.run(command, shell=True)

# Placeholder paths - you'll need to update these with actual paths
soundfont_path = 'path/to/your/soundfont.sf2'
wav_file_path = 'output.wav'